# Import Modules

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive'

/content/drive/MyDrive


In [3]:
import copy
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from mnist.data_utils import load_data

# Utils

In [4]:
def leaky_relu(z, alpha=0.01):
    """
    Implement the leaky relu activation function.
    The method takes the input z and returns the output of the function.
    Please DO NOT MODIFY the alpha value.

    Question (a)

    """
    return np.where(z>0, z, alpha*z)


def softmax(X):
    """
    Implement the softmax function.
    The method takes the input X and returns the output of the function.

    Question (a)

    """
    exps = np.exp(X)
    return exps/np.sum(exps, axis=1, keepdims=True)

def deriv_leaky_relu(x, alpha=0.01):
    """
    Implement the derivative of leaky relu activation function.
    The method takes the input z and returns the output of the function.
    Please DO NOT MODIFY the alpha value.

    Question (a)

    """
    return np.where(x>0, 1, alpha)

def load_batch(X, Y, batch_size, shuffle=True):
    """
    Generates batches with the remainder dropped.

    Do NOT modify this function
    """
    if shuffle:
        permutation = np.random.permutation(X.shape[0])
        X = X[permutation, :]
        Y = Y[permutation, :]
    num_steps = int(X.shape[0])//batch_size
    step = 0
    while step<num_steps:
        X_batch = X[batch_size*step:batch_size*(step+1)]
        Y_batch = Y[batch_size*step:batch_size*(step+1)]
        step+=1
        yield X_batch, Y_batch

# 2-Layer Neural Network

In [5]:
class TwoLayerNN:
    """ a neural network with 2 layers """

    def __init__(self, input_dim, num_hiddens, num_classes):
        """
        Do NOT modify this function.
        """
        self.input_dim = input_dim
        self.num_hiddens = num_hiddens
        self.num_classes = num_classes
        self.params = self.initialize_parameters(input_dim, num_hiddens, num_classes)

    def initialize_parameters(self, input_dim, num_hiddens, num_classes):
        """
        initializes parameters with Xavier Initialization.

        Question (b)
        - refer to https://paperswithcode.com/method/xavier-initialization for Xavier initialization

        Inputs
        - input_dim
        - num_hiddens
        - num_classes
        Returns
        - params: a dictionary with the initialized parameters.
        """
        params = {}
        W1 = np.random.randn(input_dim, num_hiddens)/np.sqrt(input_dim)
        W2 = np.random.randn(num_hiddens, num_classes)/np.sqrt(num_hiddens)
        b1 = np.zeros((num_hiddens,))
        b2 = np.zeros((num_classes,))

        params['W1'] = W1
        params['W2'] = W2
        params['b1'] = b1
        params['b2'] = b2

        return params

    def forward(self, X):
        """
        Define and perform the feed forward step of a two-layer neural network.
        Specifically, the network structue is given by

          y = softmax(LeakyReLU(X W1 + b1) W2 + b2)

        where X is the input matrix of shape (N, D), y is the class distribution matrix
        of shape (N, C), N is the number of examples (either the entire dataset or
        a mini-batch), D is the feature dimensionality, and C is the number of classes.

        Question (c)
        - ff_dict will be used to run backpropagation in backward method.

        Inputs
        - X: the input matrix of shape (N, D)

        Returns
        - y: the output of the model
        - ff_dict: a dictionary with all the fully connected units and activations.
        """
        ff_dict = {}
        z1 = np.dot(X, self.params['W1']) + self.params['b1'] # (N, num_hidden) or (B, num_hidden)
        a1 = leaky_relu(z1) # (N, num_hidden) or (B, num_hidden)
        ff_dict['z1'], ff_dict['a1'] = z1,a1

        z2 = np.dot(a1, self.params['W2']) + self.params['b2'] # (N, num_classes) or (B, num_classes)
        y = softmax(z2) # (N, num_classes) or (B, num_classes)

        ff_dict['z2'], ff_dict['y'] = z2, y
        return y, ff_dict

    def backward(self, X, Y, ff_dict):
        """
        Performs backpropagation over the two-layer neural network, and returns
        a dictionary of gradients of all model parameters.

        Question (d)

        Inputs:
         - X: the input matrix of shape (B, D), where B is the number of examples
              in a mini-batch, D is the feature dimensionality.
         - Y: the matrix of one-hot encoded ground truth classes of shape (B, C),
              where B is the number of examples in a mini-batch, C is the number
              of classes.
         - ff_dict: the dictionary containing all the fully connected units and
              activations.

        Returns:
         - grads: a dictionary containing the gradients of corresponding weights and biases.
        """
        grads = {}
        dz2 = (ff_dict['y'] - Y)  # (N,num_classes)
        grads['W2'] = np.dot(ff_dict['a1'].T, dz2) # (N,num_hidden)^T@(N,num_classes) = (num_classes,num_hidden)
        grads['b2'] = np.sum(dz2, axis=0)  # (num_classes,)
        dz1 = np.dot(dz2, self.params['W2'].T)*deriv_leaky_relu(ff_dict['z1']) # (N, num_classes)@(num_hidden, num_classes)^T * (N, num_hidden) = (N,num_hidden)
        grads['W1'] = np.dot(X.T, dz1) # (N, D)^T@(N,num_hidden) = (D, num_hidden)
        grads['b1'] = np.sum(dz1, axis=0) # (num_hidden,)

        return grads


    def compute_loss(self, Y, Y_hat):
        """
        Computes cross entropy loss.

        Do NOT modify this function.

        Inputs
            Y:
            Y_hat:
        Returns
            loss:
        """
        loss = -(1/Y.shape[0]) * np.sum(np.multiply(Y, np.log(Y_hat)))
        return loss

    def train(self, X, Y, X_val, Y_val, lr, n_epochs, batch_size, log_interval=1):
        """
        Runs mini-batch gradient descent.

        Do NOT Modify this method.

        Inputs
        - X
        - Y
        - X_val
        - Y_Val
        - lr
        - n_epochs
        - batch_size
        - log_interval
        """
        for epoch in range(n_epochs):
          for X_batch, Y_batch in load_batch(X, Y, batch_size):
              self.train_step(X_batch, Y_batch, batch_size, lr)
          if epoch % log_interval==0:
              Y_hat, ff_dict = self.forward(X)
              train_loss = self.compute_loss(Y, Y_hat)
              train_acc = self.evaluate(Y, Y_hat)
              Y_hat, ff_dict = self.forward(X_val)
              valid_loss = self.compute_loss(Y_val, Y_hat)
              valid_acc = self.evaluate(Y_val, Y_hat)
              print('epoch {:02} - train loss/acc: {:.3f} {:.3f}, valid loss/acc: {:.3f} {:.3f}'.\
                    format(epoch, train_loss, train_acc, valid_loss, valid_acc))

    def train_step(self, X_batch, Y_batch, batch_size, lr):
        """
        Updates the parameters using gradient descent.

        Do NOT Modify this method.

        Inputs
        - X_batch
        - Y_batch
        - batch_size
        - lr
        """
        _, ff_dict = self.forward(X_batch)
        grads = self.backward(X_batch, Y_batch, ff_dict)
        self.params["W1"] -= lr * grads["W1"]/batch_size
        self.params["b1"] -= lr * grads["b1"]/batch_size
        self.params["W2"] -= lr * grads["W2"]/batch_size
        self.params["b2"] -= lr * grads["b2"]/batch_size

    def evaluate(self, Y, Y_hat):
        """
        Computes classification accuracy.

        Do NOT modify this function

        Inputs
        - Y: A numpy array of shape (N, C) containing the softmax outputs,
             where C is the number of classes.
        - Y_hat: A numpy array of shape (N, C) containing the one-hot encoded labels,
             where C is the number of classes.

        Returns
            accuracy: the classification accuracy in float
        """
        classes_pred = np.argmax(Y_hat, axis=1)
        classes_gt = np.argmax(Y, axis=1)
        accuracy = float(np.sum(classes_pred==classes_gt)) / Y.shape[0]
        return accuracy

# Load MNIST

In [6]:
X_train, Y_train, X_test, Y_test = load_data()

idxs = np.arange(len(X_train))
np.random.shuffle(idxs)
split_idx = int(np.ceil(len(idxs)*0.8))
X_valid, Y_valid = X_train[idxs[split_idx:]], Y_train[idxs[split_idx:]]
X_train, Y_train = X_train[idxs[:split_idx]], Y_train[idxs[:split_idx]]
print()
print('Set validation data aside')
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', Y_train.shape)
print('Validation data shape: ', X_valid.shape)
print('Validation labels shape: ', Y_valid.shape)

MNIST data loaded:
Training data shape: (60000, 784)
Training labels shape: (60000, 10)
Test data shape: (10000, 784)
Test labels shape: (10000, 10)

Set validation data aside
Training data shape:  (48000, 784)
Training labels shape:  (48000, 10)
Validation data shape:  (12000, 784)
Validation labels shape:  (12000, 10)


# Training & Evaluation

In [ ]:
###
# Question (e)
# Tune the hyperparameters with validation data,
# and print the results by running the lines below.
###

I do grid search to find the best hyperparameter. Due to the limitation of computing resources, I skimmed relatively small region of parameter space. The range of each parameter I set in this model is as follow.

num_hiddens = [32,64,128]

lr_lst = [1e-4,1e-2,1e-1]

n_epochs_lst = [20,100,200]

batch_size_lst = [32,64,128]

In [7]:
# model instantiation
model = TwoLayerNN(input_dim=784, num_hiddens=128, num_classes=10)

In [8]:
# train the model
lr, n_epochs, batch_size = 0.1, 200, 128
model.train(X_train, Y_train, X_valid, Y_valid, lr, n_epochs, batch_size)

epoch 00 - train loss/acc: 0.321 0.909, valid loss/acc: 0.337 0.904
epoch 01 - train loss/acc: 0.263 0.925, valid loss/acc: 0.281 0.921
epoch 02 - train loss/acc: 0.224 0.938, valid loss/acc: 0.244 0.932
epoch 03 - train loss/acc: 0.195 0.946, valid loss/acc: 0.218 0.939
epoch 04 - train loss/acc: 0.172 0.952, valid loss/acc: 0.198 0.944
epoch 05 - train loss/acc: 0.153 0.957, valid loss/acc: 0.180 0.949
epoch 06 - train loss/acc: 0.138 0.961, valid loss/acc: 0.166 0.951
epoch 07 - train loss/acc: 0.126 0.965, valid loss/acc: 0.158 0.955
epoch 08 - train loss/acc: 0.117 0.968, valid loss/acc: 0.150 0.957
epoch 09 - train loss/acc: 0.108 0.969, valid loss/acc: 0.142 0.959
epoch 10 - train loss/acc: 0.100 0.973, valid loss/acc: 0.135 0.961
epoch 11 - train loss/acc: 0.092 0.975, valid loss/acc: 0.129 0.963
epoch 12 - train loss/acc: 0.086 0.977, valid loss/acc: 0.122 0.964
epoch 13 - train loss/acc: 0.082 0.978, valid loss/acc: 0.123 0.964
epoch 14 - train loss/acc: 0.075 0.980, valid lo

In [9]:
# evalute the model on test data
Y_hat, _ = model.forward(X_test)
test_loss = model.compute_loss(Y_test, Y_hat)
test_acc = model.evaluate(Y_test, Y_hat)
print("Final test loss = {:.3f}, acc = {:.3f}".format(test_loss, test_acc))

Final test loss = 0.089, acc = 0.978
